In [ ]:
import streamlit as st
import pandas as pd
from snowflake.snowpark.context import get_active_session
import plotly.express as px
import altair as alt

# Set wide layout
st.set_page_config(layout="wide")

# Widen the Streamlit canvas
st.markdown("""
    <style>
        .main .block-container {
            max-width: 95%;
            padding-top: 1rem;
            padding-right: 2rem;
            padding-left: 2rem;
        }
    </style>
""", unsafe_allow_html=True)

# Connect to Snowflake
session = get_active_session()

# Page navigation in sidebar
# st.sidebar.title("Navigation")
# page = st.sidebar.radio("Select Page", ["Candidate Profiles", "Resume Ranking"])

st.sidebar.title("Navigation")

# Initialize the page in session_state
if "page" not in st.session_state:
    st.session_state.page = "Candidate Profiles"  # default page

if st.sidebar.button("Candidate Profiles"):
    st.session_state.page = "Candidate Profiles"

if st.sidebar.button("Resume Ranking"):
    st.session_state.page = "Resume Ranking"

# Common function to fetch candidate data
@st.cache_data
def fetch_candidate_data():
    query = """
    SELECT FULL_NAME, PHONE_NUMBER, EMAIL, LOCATION, JOB_TITLE, JOB_LEVEL, WORK_EXPERIENCE_ROLE,
           RECENT_WORK_EXPERIENCE, TECHNICAL_SKILLS, YEARS_OF_EXPERIENCE, DEGREE, FIELD_OF_STUDY,
           UNIVERSITY_NAME, CERTIFICATION, LANGUAGES_SPOKEN, CONFIDENCE_SCORE, VALIDATION_STATUS, 
           APPLICATION_STATUS, REMARK
    FROM CANDIDATE_PROFILES
    """
    return session.sql(query).to_pandas()

# Page 1: Candidate Profiles
if st.session_state.page == "Candidate Profiles":
    st.title("👤 Candidate Profiles")
    # Fetch data
    df = fetch_candidate_data()
    
    # Count how many candidates need validation
    num_validation = (df["VALIDATION_STATUS"] == "Need Validation").sum()

    # Show it in Streamlit sidebar
    st.sidebar.markdown(f"""
    <span style='font-size:18px;'>🚨 Validation Needed: <strong style='font-size:20px; font-weight:bold; color:#d62828;'>{num_validation}</strong></span>
    """, unsafe_allow_html=True)

    # === Sidebar Filters ===
    st.sidebar.header("Filters")
    job_titles = df['JOB_TITLE'].dropna().unique()
    job_levels = df['JOB_LEVEL'].dropna().unique()
    all_skills = sorted({skill.strip() for skills in df['TECHNICAL_SKILLS'].fillna('').tolist() for skill in skills.split(',') if skill.strip()})
    min_exp = 1
    max_exp = int(df['YEARS_OF_EXPERIENCE'].max())

    # Unique validation statuses from the dataframe
    validation_options = ["All", "Validated", "Need Validation", "Pass Score"]

    # Unique validation statuses from the dataframe
    application_options = ["All", "Applied", "Under Review", "Shortlisted", "Interview", "Rejected", "Hired"]

    selected_job_titles = st.sidebar.multiselect("Job Title", job_titles)
    selected_job_levels = st.sidebar.multiselect("Job Level", job_levels)
    selected_skills = st.sidebar.multiselect("Technical Skills", all_skills)
    selected_exp = st.sidebar.slider("Minimum Years of Experience", min_value=min_exp, max_value=max_exp, value=min_exp)

    # Sidebar filter for validation
    selected_validation = st.sidebar.selectbox("Validation Status", validation_options)

    # Sidebar filter for validation
    selected_application = st.sidebar.selectbox("Application Status", application_options)

    # === Apply Filters ===
    filtered_df = df.copy()
    if selected_job_titles:
        filtered_df = filtered_df[filtered_df['JOB_TITLE'].isin(selected_job_titles)]
    if selected_job_levels:
        filtered_df = filtered_df[filtered_df['JOB_LEVEL'].isin(selected_job_levels)]
    filtered_df = filtered_df[filtered_df['YEARS_OF_EXPERIENCE'] >= selected_exp]
    
    if selected_skills:
        filtered_df = filtered_df[
            filtered_df['TECHNICAL_SKILLS'].apply(
                lambda s: all(skill in s for skill in selected_skills)
            )
        ]
        
    #Filter validation status
    if selected_validation != "All":
        filtered_df = filtered_df[filtered_df['VALIDATION_STATUS'] == selected_validation]

    #Filter application status
    if selected_application != "All":
        filtered_df = filtered_df[filtered_df['APPLICATION_STATUS'] == selected_application]

    # === Display Cards ===
    #st.write("# Candidate Resume Profile")

    # Donut Chart: Job Title Distribution
    job_title_counts = filtered_df['JOB_TITLE'].value_counts().reset_index()
    job_title_counts.columns = ['Job Title', 'Count']

    # Bar Chart: Job Levels Distribution
    job_level_counts = filtered_df['JOB_LEVEL'].value_counts().reset_index()
    job_level_counts.columns = ['Job Level', 'Count']

    
    if not job_title_counts.empty:
        fig = px.pie(job_title_counts, names='Job Title', values='Count',
                    title='Distribution of Job Titles',
                    hole=0.4,
                    color_discrete_sequence=px.colors.qualitative.Pastel)

        fig.update_traces(textinfo='label+value', textposition='outside', pull=[0.05] * len(job_title_counts))
        fig.update_layout(margin=dict(t=50, b=10, l=0, r=0), height=300)
        st.plotly_chart(fig, use_container_width=True)

        # Create a bar chart using Altair with color by 'Job Title'
        chart = alt.Chart(job_level_counts).mark_bar().encode(
            x='Job Level',
            y='Count',
            color='Job Level'  # Color bars based on the 'Job Level' column
        ).properties(
            title='Distribution of Job Levels'  # Set the title for the chart
        )
        
        # Display the chart in Streamlit
        st.altair_chart(chart, use_container_width=True)
        
    else:
        st.info("No data available to show in the donut chart based on current filters.")
        
    #Display the filtered dataset
    st.write(filtered_df)
    
    for idx, row in filtered_df.iterrows():
        skill_list = [s.strip() for s in row['TECHNICAL_SKILLS'].split(',') if s.strip()]
        
        job_title_colors = {
            "Data Scientist": "#b3e5fc", "Data Analyst": "#ffccbc", "BI Consultant": "#c8e6c9", "Data Scientist ": "#f0f4c3"
        }
        job_level_colors = {
            "Junior": "#ffe082", "Mid-level": "#ffeb3b", "Senior": "#f57c00"
        }

        application_status_colors = {
            "Applied": "#e3f2fd",        # Light Blue
            "Under Review": "#fff9c4",   # Light Yellow
            "Shortlisted": "#c8e6c9",    # Light Green
            "Interview": "#b3e5fc",      # Sky Blue
            "Rejected": "#ffcdd2",       # Light Red
            "Hired": "#dcedc8"           # Light Lime Green
        }
        
        job_title_color = job_title_colors.get(row['JOB_TITLE'], "#e0f7fa")
        job_level_color = job_level_colors.get(row['JOB_LEVEL'], "#fce4ec")
        application_status_color = application_status_colors.get(row['APPLICATION_STATUS'], "#e0e0e0")
        
        with st.container():
            st.markdown(
                f"""
                <div style='
                    border: 1px solid #ddd;
                    border-radius: 8px;
                    padding: 5px 15px;
                    margin-bottom: 10px;
                    background-color: #fff;
                '>
                """,
                unsafe_allow_html=True
            )
            
            inner_cols = st.columns([3, 3, 2, .5])
            with inner_cols[0]:
                st.markdown(f"### {row['FULL_NAME']}")
                st.markdown(f"**Experience:** {row['YEARS_OF_EXPERIENCE']} years")
                
                # Determine the badge to show based on validation status
                if row['VALIDATION_STATUS'] == "Need Validation":
                    validation_badge = "<span style='background-color:#f8d7da; color:#c62828; padding:4px 8px; border-radius:8px; font-size:12px; margin-left:6px'>❌ Need Validation</span>"
                elif row['VALIDATION_STATUS'] == "Validated":
                    validation_badge = "<span style='background-color:#d0f0c0; color:#2e7d32; padding:4px 8px; border-radius:8px; font-size:12px; margin-left:6px'>✅ Validated</span>"
                else:
                    validation_badge = "<span style='background-color:#e3f2fd; color:#0277bd; padding:4px 8px; border-radius:8px; font-size:12px; margin-left:6px'>ℹ️ Pass Score</span>"
                
                # Then insert it into your HTML
                st.markdown(
                    f"""
                    <div style="margin-top: 1px;">
                        <span style='background-color:{job_title_color}; color:#006064; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px'>
                            {row['JOB_TITLE']}
                        </span>
                        <span style='background-color:{job_level_color}; color:#880e4f; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px'>
                            {row['JOB_LEVEL']}
                        </span>
                        {validation_badge}
                    </div>
                    """,
                    unsafe_allow_html=True
                )

            
            with inner_cols[1]:
                st.markdown(f"**Skills:**")
                skill_colors = ["#a5d6a7", "#c5cae9", "#ffcc80", "#ffab91", "#80deea", "#f48fb1"]
                skill_badges = " ".join([
                    f"<span style='background-color:{skill_colors[i % len(skill_colors)]}; color:#2e7d32; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px'>{s}</span>"
                    for i, s in enumerate(skill_list)
                ])
                st.markdown(f"<div style='margin-top:10px; display: flex; flex-wrap: wrap'>{skill_badges}</div>", unsafe_allow_html=True)
            
            with inner_cols[2]:
                st.markdown(f"**Application Status:**")
                st.markdown(
                    f"""
                    <div style="margin-top: 10px;">
                        <span style='background-color:{application_status_color}; color:#006064; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px'>
                            {row['APPLICATION_STATUS']}
                        </span>
                    </div>
                    """,
                    unsafe_allow_html=True
                )
            
            with inner_cols[3]:
                edit_key = f"edit_{idx}"
                if st.button("Edit", key=edit_key):
                    st.session_state[f"edit_card_{idx}"] = not st.session_state.get(f"edit_card_{idx}", False)

            st.markdown("</div>", unsafe_allow_html=True)

        def update_candidate_profile(name, updated_values):
            set_clause = ", ".join([
                f"{col} = '{val}'" if isinstance(val, str) else f"{col} = {val}"
                for col, val in updated_values.items()
            ])
            query = f"""
            UPDATE CANDIDATE_PROFILES
            SET {set_clause}
            WHERE FULL_NAME = '{name}'
            """
            session.sql(query).collect()

        if st.session_state.get(f"edit_card_{idx}", False):
            with st.expander(f"Edit: {row['FULL_NAME']}", expanded=True):
                new_job_title = st.text_input("Job Title", value=row['JOB_TITLE'], key=f"title_{idx}")
                new_job_level = st.text_input("Job Level", value=row['JOB_LEVEL'], key=f"level_{idx}")
                new_experience = st.number_input("Years of Experience", min_value=0, value=int(row['YEARS_OF_EXPERIENCE']), key=f"exp_{idx}")
                new_skills = st.text_input("Technical Skills (comma-separated)", value=row['TECHNICAL_SKILLS'], key=f"skills_{idx}")

                # Dropdown for Validation Status
                validation_status_options = ["Need Validation", "Validated", "Pass Score"]
                new_validation_status = st.selectbox(
                    "Validation Status", 
                    options=validation_status_options,
                    index=validation_status_options.index(row['VALIDATION_STATUS']) if row['VALIDATION_STATUS'] in validation_status_options else 0,
                    key=f"vali_status_{idx}"
                )
                
                # Dropdown for Application Status
                application_status_options = ["Applied ", "Under Review", "Shortlisted", "Interview", "Rejected", "Hired"]
                new_application_status = st.selectbox(
                    "Application Status",
                    options=application_status_options,
                    index=application_status_options.index(row['APPLICATION_STATUS']) if row['APPLICATION_STATUS'] in application_status_options else 0,
                    key=f"appli_status_{idx}"
                )
        
                new_remark = st.text_input("Validation Remark", value=row['REMARK'], key=f"remark_{idx}")
        
                
                if st.button("Submit Changes", key=f"submit_{idx}"):
                    updated_values = {
                        'JOB_TITLE': new_job_title,
                        'JOB_LEVEL': new_job_level,
                        'YEARS_OF_EXPERIENCE': new_experience,
                        'TECHNICAL_SKILLS': new_skills,
                        'VALIDATION_STATUS': new_validation_status,
                        'APPLICATION_STATUS': new_application_status,
                        'REMARK': new_remark
                    }
                    update_candidate_profile(row['FULL_NAME'], updated_values)
                    st.success(f"{row['FULL_NAME']}'s profile updated!")
                    st.cache_data.clear()
                    st.rerun()

        st.markdown("</div>", unsafe_allow_html=True)


# Page 2: Resume Ranking
elif st.session_state.page == "Resume Ranking":
    st.title("📊 Resume Ranking")
        # SQL query to fetch candidate data from Snowflake
    @st.cache_data
    def fetch_data():
        query = """
        SELECT FULL_NAME, PHONE_NUMBER, EMAIL, LOCATION, JOB_TITLE, JOB_LEVEL, TECHNICAL_SKILLS, YEARS_OF_EXPERIENCE 
        FROM CANDIDATE_PROFILES
        """
        return session.sql(query).to_pandas()
    
    user_data = fetch_data()
    
    # Job Roles
    job_roles = pd.DataFrame({
        'job_role': ['Data Analyst', 'BI Consultant', 'Data Scientist', 'Data Engineer'],
        'base_weight': [70, 80, 85, 90]
    })
    
    # Job Levels
    job_levels = pd.DataFrame({
        'job_level': ['Junior', 'Mid-level', 'Senior', 'Expert'],
        'level_weight': [0.5, 1, 1.5, 2]
    })
    
    # Skill Weights
    skills = pd.DataFrame({
        'skill': [
            'AWS', 'Apache Airflow', 'Apache Kafka', 'Apache Spark', 'DAX', 'Excel', 'Hadoop', 
            'NLP', 'Power BI', 'Python', 'R', 'SQL', 'Tableau', 'TensorFlow'
        ],
        'Data Analyst': [5, 0, 0, 0, 20, 15, 0, 0, 25, 15, 0, 20, 20, 0],
        'BI Consultant': [0, 10, 5, 5, 20, 10, 10, 5, 25, 15, 10, 15, 25, 10],
        'Data Scientist': [10, 15, 15, 10, 25, 15, 10, 10, 15, 20, 20, 20, 15, 30],
        'Data Engineer': [20, 20, 25, 25, 10, 10, 20, 15, 10, 20, 10, 25, 5, 15]
    })
    
    # Experience weights
    experience = pd.DataFrame({
        'years_of_experience': [1, 2, 3, 4, 5, 6, 7],
        'experience_weight': [10, 15, 20, 25, 30, 35, 40]
    })
    
    # Helper Functions
    def get_weight(df, column, value, weight_col):
        row = df[df[column] == value]
        return row[weight_col].values[0] if not row.empty else 0
    
    def get_role_specific_skill_weight(skill, job_role):
        row = skills[skills['skill'] == skill]
        return row[job_role].values[0] if not row.empty else 0
    
    def calculate_weight(job_role, job_level, selected_skills, years_exp):
        role_weight = get_weight(job_roles, 'job_role', job_role, 'base_weight')
        level_weight = get_weight(job_levels, 'job_level', job_level, 'level_weight')
        skill_weight = sum([get_role_specific_skill_weight(s, job_role) for s in selected_skills])
        exp_weight = get_weight(experience, 'years_of_experience', years_exp, 'experience_weight')
        total = role_weight + (role_weight * level_weight) + skill_weight + exp_weight
        return round(total, 2)
    
    def candidate_score(row, selected_job, selected_level, selected_skills, selected_exp):
        role_weight = get_weight(job_roles, 'job_role', selected_job, 'base_weight')
        candidate_level = row['JOB_LEVEL']
        candidate_exp = row['YEARS_OF_EXPERIENCE']
        level_weight = get_weight(job_levels, 'job_level', candidate_level, 'level_weight')
        candidate_skills = set(map(str.strip, row['TECHNICAL_SKILLS'].split(',')))
        matched_skills = set(selected_skills).intersection(candidate_skills)
        skill_weight = sum([get_role_specific_skill_weight(s, selected_job) for s in matched_skills])
        exp_weight = get_weight(experience, 'years_of_experience', candidate_exp, 'experience_weight')
        total = role_weight + (role_weight * level_weight) + skill_weight + exp_weight
        return round(total)
    
    # --- Streamlit App ---
    #st.title("📊 ATS Resume Ranking System")
    min_exp = int(experience['years_of_experience'].min())
    max_exp = int(experience['years_of_experience'].max())
    
    # Inputs
    selected_job = st.sidebar.multiselect("Select Job Role", job_roles['job_role'])
    #all_levels = job_levels['job_level'].tolist()
    selected_levels = st.sidebar.multiselect("Select Job Level(s)", job_levels['job_level'])
    selected_skills = st.sidebar.multiselect("Select Required Skills", skills['skill'])
    selected_exp = st.sidebar.slider("Select Minimum Years of Experience", min_value=min_exp, max_value=max_exp, value=min_exp)
    top_n = st.sidebar.selectbox("Select number of top candidates to display", ['All', 1, 2, 3, 4, 5])
    
    # Filter Candidates
    filtered_data = user_data.copy()
    
    if selected_job:
        filtered_data = filtered_data[filtered_data['JOB_TITLE'].isin(selected_job)]
    
    # Filter by selected job levels
    if selected_levels:
        filtered_data = filtered_data[filtered_data['JOB_LEVEL'].isin(selected_levels)]
    
    # Filter by skills
    filtered_data = filtered_data[
        filtered_data['TECHNICAL_SKILLS'].apply(
            lambda s: set(selected_skills).issubset(set(map(str.strip, s.split(','))))
        )
    ]
    
    # Filter by experience
    filtered_data = filtered_data[
        filtered_data['YEARS_OF_EXPERIENCE'] >= selected_exp
    ]
    
    # Check if filtered data is empty
    if filtered_data.empty:
        st.warning("No candidates match the selected criteria.")
    else:
        # Use default level for scoring if 'All' levels were selected
        selected_level_for_score = selected_levels[0] if selected_levels else 'Junior'
        selected_job_for_score = selected_job[0] if selected_job else 'Data Analyst'
    
        baseline_weight = calculate_weight(selected_job_for_score, selected_level_for_score, selected_skills, selected_exp)
        st.markdown(f"### Baseline Competency Weight: `{baseline_weight}`")
    
        # Score all candidates using selected inputs
        filtered_data['competency_weight'] = filtered_data.apply(
            lambda row: candidate_score(row, selected_job_for_score, selected_level_for_score, selected_skills, selected_exp),
            axis=1
        )
    
        filtered_data['status'] = filtered_data['competency_weight'].apply(
            lambda x: "✓" if x >= baseline_weight else "✗"
        )
    
        # Sort and select top candidates
        ranked_users = filtered_data.sort_values(by='competency_weight', ascending=False)
        if top_n != 'All':
            ranked_users = ranked_users.head(int(top_n))
    
        # Styling
        def highlight_status(val):
            color = 'green' if val == '✓' else 'red'
            return f'color: {color}; font-weight: bold;'
    
        styled_df = ranked_users[[
            'FULL_NAME', 'PHONE_NUMBER', 'EMAIL', 'LOCATION', 'JOB_TITLE', 'JOB_LEVEL', 
            'TECHNICAL_SKILLS', 'YEARS_OF_EXPERIENCE', 'competency_weight', 'status'
        ]].style.applymap(highlight_status, subset=['status'])
    
        st.subheader("🏅 Ranked Candidates")
        st.dataframe(styled_df, use_container_width=True)

        #Resume Card Profile 
        original_df = styled_df.data  # Get the original DataFrame
        
        for idx, row in original_df.iterrows():
        
            #getting the skills
            skill_list = [s.strip() for s in row['TECHNICAL_SKILLS'].split(',') if s.strip()]
            
            #define color for each titles and job levels
            job_title_colors = {"Data Scientist": "#b3e5fc", "Data Analyst": "#ffccbc", "BI Consultant": "#c8e6c9", "Data Scientist ": "#f0f4c3"}
            job_level_colors = {"Junior": "#ffe082", "Mid-level": "#ffeb3b", "Senior": "#f57c00"}
        
            #default color 
            job_title_color = job_title_colors.get(row['JOB_TITLE'], "#e0f7fa")
            job_level_color = job_level_colors.get(row['JOB_LEVEL'], "#fce4ec")
        
            #set border between each resume
            with st.container():
                st.markdown(
                    f"""
                    <div style='
                        border: 1px solid #ddd;
                        border-radius: 8px;
                        padding: 5px 15px;
                        margin-bottom: 10px;
                        background-color: #fff;
                    '>
                    """,
                    unsafe_allow_html=True
                )
        
                #name, experience, job title, job level 
                inner_cols = st.columns([3, 3, 2, .5])
                with inner_cols[0]:
                    st.markdown(
                        """
                        <div style="margin: 20px;">
                            <h3>{}</h3>
                            <p><strong>Experience:</strong> {} years</p>
                            <div style="margin-bottom: 15px;">
                                <span style="background-color:{}; color:#006064; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px">
                                    {}
                                </span>
                                <span style="background-color:{}; color:#880e4f; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px">
                                    {}
                                </span>
                            </div>
                        </div>
                        """.format(
                            row['FULL_NAME'], 
                            row['YEARS_OF_EXPERIENCE'], 
                            job_title_color, 
                            row['JOB_TITLE'], 
                            job_level_color, 
                            row['JOB_LEVEL']
                        ),
                        unsafe_allow_html=True  
                    )
        
        
                #technical skills 
                with inner_cols[1]:
                    st.markdown(f"**Skills:**")
                    skill_colors = ["#a5d6a7", "#c5cae9", "#ffcc80", "#ffab91", "#80deea", "#f48fb1"]
                    skill_badges = " ".join([
                        f"<span style='background-color:{skill_colors[i % len(skill_colors)]}; color:#2e7d32; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px'>{s}</span>"
                        for i, s in enumerate(skill_list)
                    ])
                    st.markdown(f"<div style='margin-top:10px; display: flex; flex-wrap: wrap'>{skill_badges}</div>", unsafe_allow_html=True)
                    st.markdown("</div>", unsafe_allow_html=True)
        
        
                with inner_cols[2]:
                    st.markdown(f"**Competency Weight:**")
                    st.markdown(
                        f"""
                        <span style='
                            background-color:#d0f0c0; 
                            color:#2e7d32; 
                            padding:8px 16px; 
                            border-radius:12px; 
                            font-size:16px; 
                            font-weight:bold; 
                        '>
                            {row['competency_weight']}
                        </span>
                        """,
                        unsafe_allow_html=True
                    )
        
        
        
                st.markdown("</div>", unsafe_allow_html=True)